# 使用TorchScript导出PyTorch模型
本节将使用tracing和scripting两种方式将PyTorch模型转换为TorchScript模型

## 使用TorchScript导出模型——Tracing
Tracing方法需要跟踪模型对输入变量所进行的操作，因此我们需要生成一个“伪变量”。由于Tracing过程并不是实际推理过程，因此“伪变量”可以是没有实际意义的随机变量，只需要其格式满足模型的输入要求即可。在本示例中，模型的输入要求是一个尺寸为（N，1，28，28）的tensor，其中N是batch数。因此我们使用torch.ones生成一个符合该尺寸的tensor，该tensor的内容全部为1。

需要注意的是，在完成模型的初始化与参数加载后，除了为模型设置.eval()属性，还需要为模型中所有的参数的request_grad属性设置为False，表示不需要计算该参数的梯度。如果不设置该参数，为了保证在执行TorchScript模型的推理过程中不计算梯度，需要添加with torch_no_grad()对模型推理过程进行包裹，避免计算梯度。

我们将模型和“伪变量”传入到torch.jit.trace()方法中，生成TorchScript模型。可通过查看模型图或模型代码详细了解tracing模型所记录的内容。

随后，通过torch.jit.load保存该模型，并在不同的开发环境中（如C++）加载并使用该模型进行推理。


## 使用TorchScript导出模型——Scripting
Scripting方式与Tracing方式的区别在于其直接检测nn.Module中的模型代码，而无需传入一个“伪变量”。因此，使用scripting方式转换PyTorch模型时不再需要初始化一个随机变量。只需直接调用torch.jit.script()方法即可。

TorchScript模型的导入与推理与Tracing方式相同。

**注意：根据TorchScript的模型图和模型代码可知，TorchScript记录了模型在正向传播过程中的操作，因此在加载TorchScript模型时不再需要初始化模型。**

In [ ]:
# Download Pretrained Model
!gdown --id 1omW_gpSpedWjo5GahxzksNW-J8MGrfdk --output convnet.pth

# Download Testing Image
!gdown --id 1A47zfadA6oXGhMvI-iCRj6CWB7rS8dWz --output digit_image.jpg

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import numpy as np
from PIL import Image

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.cn1 = nn.Conv2d(1, 16, 3, 1)
        self.cn2 = nn.Conv2d(16, 32, 3, 1)
        self.dp1 = nn.Dropout2d(0.10)
        self.dp2 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(4608, 64) # 4608 is basically 12 X 12 X 32
        self.fc2 = nn.Linear(64, 10)
 
    def forward(self, x):
        x = self.cn1(x)
        x = F.relu(x)
        x = self.cn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dp1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)
        x = self.fc2(x)
        op = F.log_softmax(x, dim=1)
        return op
    
model = ConvNet()

In [ ]:
PATH_TO_MODEL = "./convnet.pth"
model.load_state_dict(torch.load(PATH_TO_MODEL, map_location="cpu"))

In [ ]:
model.eval()
for p in model.parameters():
    p.requires_grad_(False)

In [ ]:
demo_input = torch.ones(1, 1, 28, 28)
traced_model = torch.jit.trace(model, demo_input) # tracing

scripted_model = torch.jit.script(model) # scripting

查看记录的模型图

In [ ]:
traced_model.graph

In [ ]:
scripted_model.graph

查看记录的模型代码

In [ ]:
print(traced_model.code)

In [ ]:
print(scripted_model.code)

In [45]:
torch.jit.save(traced_model, 'traced_convnet.pt') 
torch.jit.save(scripted_model, 'scripted_convnet.pt') 

In [47]:
loaded_traced_model = torch.jit.load('traced_convnet.pt') 
loaded_scripted_model = torch.jit.load('scripted_convnet.pt')

In [ ]:
image = Image.open("./digit_image.jpg")

In [ ]:
image

In [ ]:
def image_to_tensor(image):
    gray_image = transforms.functional.to_grayscale(image)
    resized_image = transforms.functional.resize(gray_image, (28, 28))
    input_image_tensor = transforms.functional.to_tensor(resized_image)
    input_image_tensor_norm = transforms.functional.normalize(input_image_tensor, (0.1302,), (0.3069,))
    return input_image_tensor_norm

In [ ]:
input_tensor = image_to_tensor(image)

In [48]:
loaded_traced_model(input_tensor.unsqueeze(0)) # 使用tracing模型进行推理

tensor([[-9.3505e+00, -1.2089e+01, -2.2391e-03, -8.9248e+00, -9.8197e+00,
         -1.3350e+01, -9.0460e+00, -1.4492e+01, -6.3023e+00, -1.2283e+01]])

In [49]:
loaded_scripted_model(input_tensor.unsqueeze(0))

tensor([[-9.3505e+00, -1.2089e+01, -2.2391e-03, -8.9248e+00, -9.8197e+00,
         -1.3350e+01, -9.0460e+00, -1.4492e+01, -6.3023e+00, -1.2283e+01]])

In [ ]:
model(input_tensor.unsqueeze(0)) # 使用原模型进行推理